In [5]:
# 安装Python库rpy2
!pip install rpy2

# 安装R包INLA及其依赖包
!R -e "install.packages(c('sp', 'fmesher'), repos=c('http://cran.r-project.org'))"
!R -e "install.packages('INLA', repos='https://inla.r-inla-download.org/R/stable')"


R version 4.3.3 (2024-02-29) -- "Angel Food Cake"
Copyright (C) 2024 The R Foundation for Statistical Computing
Platform: x86_64-pc-linux-gnu (64-bit)

R is free software and comes with ABSOLUTELY NO WARRANTY.
You are welcome to redistribute it under certain conditions.
Type 'license()' or 'licence()' for distribution details.

  Natural language support but running in an English locale

R is a collaborative project with many contributors.
Type 'contributors()' for more information and
'citation()' on how to cite R or R packages in publications.

Type 'demo()' for some demos, 'help()' for on-line help, or
'help.start()' for an HTML browser interface to help.
Type 'q()' to quit R.

> install.packages(c('sp', 'fmesher'), repos=c('http://cran.r-project.org'))
Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)
trying URL 'http://cran.r-project.org/src/contrib/sp_2.1-3.tar.gz'
Content type 'application/x-gzip' length 1244605 bytes (1.2 MB)
downloaded 1.2 MB


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from rpy2 import robjects as ro
from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter

# 激活Pandas DataFrame到R data.frame的自动转换
pandas2ri.activate()

# 加载INLA库
ro.r('library(INLA)')

def run_inla_model(formula, data, model_name):
    with localconverter(ro.default_converter + pandas2ri.converter):
        r_df = ro.conversion.py2rpy(data)

    # 在R中创建滞后变量
    if 'lag' in formula:
        formula_parts = formula.split('~')
        lags = []
        for term in formula_parts[1].split('+'):
            if 'lag' in term:
                lag_term = term.strip().split('_')[-1]  # 例如 'lag1'
                lag = int(''.join(filter(str.isdigit, lag_term)))  # 提取数字部分
                lags.append(lag)
        for lag in lags:
            ro.r(f'r_df$HeatCount_lag{lag} <- c({",".join(["NA"]*lag)}, head(r_df$HeatCount, -{lag}))')

    inla_call = f"""
    result <- inla(formula = {formula}, data = r_df, family = 'gaussian',
                   control.predictor = list(compute = TRUE))
    """
    ro.r(inla_call)
    ro.r(f'print(summary(result), digits = 3)')
    mlik = ro.r('result$mlik')[0]
    print(f"模型: {model_name}")
    print(f"边际对数似然: {mlik}")
    print()
    return mlik

# 加载数据
df = pd.read_csv('/content/drive/My Drive/filtered_summer_data.csv')

# 根据STATEname进行分组
state_groups = df.groupby('STATEname')

# 存储模型结果
model_results = []

for state, group_data in state_groups:
    # 为每个州存储模型边际对数似然值
    mliks = []
    models = []

    print(f"处理州：{state}")
    # 运行不包含滞后变量的模型
    base_formula = "SentimentScore ~ HeatCount"
    print("运行不包含滞后变量的模型")
    base_mlik = run_inla_model(base_formula, group_data, "基础模型")
    mliks.append(base_mlik)
    models.append("基础模型")

    # 运行包含不同滞后天数的模型
    for lag_days in range(1, 8):  # 比较1到7天的滞后
        lagged_formula = "SentimentScore ~ HeatCount"
        for i in range(1, lag_days + 1):
            lagged_formula += f" + HeatCount_lag{i}"  # 每天滞后
        model_name = f"{state}_滞后{i}天模型"
        print(f"运行包含滞后{lag_days}天的模型")
        mlik = run_inla_model(lagged_formula, group_data, model_name)
        mliks.append(mlik)
        models.append(model_name)

    # 绘制边际对数似然值变化趋势
    plt.figure(figsize=(12, 6))
    plt.plot(models, mliks, marker='o')
    plt.xticks(rotation=45)
    plt.ylabel('边际对数似然值')
    plt.title(f'{state} - 模型边际对数似然值变化趋势')
    plt.show()

    model_results.append((state, models, mliks))

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


全国数据集的路径是/content/drive/My Drive/filtered_summer_data.csv

In [7]:
import pandas as pd
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri
from rpy2.robjects.packages import importr

# 自动转换Pandas DataFrame为R的data.frame
pandas2ri.activate()

# 加载R的INLA库
inla = importr('INLA')
base = importr('base')

In [ ]:

import pandas as pd

# 加载数据
data_path = "/content/drive/My Drive/filtered_summer_data.csv"
df = pd.read_csv(data_path)

In [ ]:
import pandas as pd
from rpy2 import robjects as ro
from rpy2.robjects import pandas2ri
from rpy2.rinterface_lib.embedded import RRuntimeError

# 激活Pandas DataFrame到R data.frame的自动转换
pandas2ri.activate()

# 尝试加载INLA库,捕获任何错误
try:
    inla = ro.r('library(INLA)')
except RRuntimeError as e:
    print(f"Failed to load INLA library in R: {e}")

def run_inla_model(formula, data):
    """
    使用INLA运行贝叶斯层次模型。
    :param formula: R风格的模型公式字符串,使用反引号括住列名。
    :param data: 数据集,Pandas DataFrame格式。
    """
    try:
        # 将Pandas DataFrame转换为R的data.frame
        r_df = pandas2ri.py2rpy(data)
        # 设置R全局环境中的数据框
        ro.globalenv['r_df'] = r_df
        # 运行INLA模型
        model_fit = ro.r(f"""
            inla(formula = '{formula}', data = r_df, family = 'gaussian', control.predictor = list(compute = TRUE))
        """)
        # 打印模型摘要
        print(ro.r('summary')(model_fit))
    except RRuntimeError as e:
        print(f"Error running INLA model: {e}")

# 加载数据
df = pd.read_csv('/content/drive/My Drive/filtered_summer_data.csv')

# 将列名转换为小写,避免大小写问题
df.columns = [col.lower() for col in df.columns]

# 定义模型公式,使用反引号括住列名
model_formulas = {
    "Model 1": "`sentimentscore` ~ `heatcount`"
}

# 运行模型
for model_name, formula in model_formulas.items():
    print(f"Running {model_name} with formula: {formula}")
    run_inla_model(formula, df)
    print("\n" + "="*50 + "\n")



 - See www.r-inla.org/contact-us for how to get help.
 - List available models/likelihoods/etc with inla.list.models()
 - Use inla.doc(<NAME>) to access documentation



Running Model 1 with formula: `sentimentscore` ~ `heatcount`


/usr/local/lib/python3.10/dist-packages/rpy2/robjects/pandas2ri.py:55: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for name, values in obj.iteritems():



 *** inla.core.safe:  The inla program failed, but will rerun in case better initial values may help. try=1/1 


  object 'y...fake' not found
The inla program failed and the maximum number of tries has been reached.



Error running INLA model: Error in inla.core.safe(formula = formula, family = family, contrasts = contrasts,  : 
  object 'y...fake' not found
The inla program failed and the maximum number of tries has been reached.





这部分现在是有问题的，无法查看全国的数据，我们返回使用加州的数据进行实验

In [8]:
import pandas as pd
import matplotlib.pyplot as plt
from rpy2 import robjects as ro
from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter

# 激活Pandas DataFrame到R data.frame的自动转换
pandas2ri.activate()

# 加载INLA库
ro.r('library(INLA)')

def run_inla_model(formula, data, model_name):
    with localconverter(ro.default_converter + pandas2ri.converter):
        r_df = ro.conversion.py2rpy(data)

    # 在R中创建滞后变量(如果需要)
    if 'lag' in formula:
        formula_parts = formula.split('~')
        lags = []
        for term in formula_parts[1].split('+'):
            if 'lag' in term:
                lag_term = term.strip().split('_')[-1]  # e.g., 'lag1'
                lag = int(''.join(filter(str.isdigit, lag_term)))  # 提取数字部分
                lags.append(lag)
        for lag in lags:
            ro.r(f'r_df$HeatCount_lag{lag} <- c({",".join(["NA"]*lag)}, head(r_df$HeatCount, -{lag}))')

    inla_call = f"""
    result <- inla(formula = {formula}, data = r_df, family = 'gaussian',
                   control.predictor = list(compute = TRUE))
    """
    ro.r(inla_call)
    ro.r(f'print(summary(result), digits = 3)')
    mlik = ro.r('result$mlik')[0]
    print(f"模型: {model_name}")
    print(f"边际对数似然: {mlik}")
    print()
    return mlik

# 加载数据
df = pd.read_csv('/content/TestData_California.csv')

# 存储模型边际对数似然值
mliks = []
models = []

# 运行不包含滞后变量的模型
base_formula = "SentimentScore ~ HeatCount"
print("运行不包含滞后变量的模型")
base_mlik = run_inla_model(base_formula, df, "基础模型")
mliks.append(base_mlik)
models.append("基础模型")

# 运行包含不同滞后天数的模型
for lag_days in range(1, 8):  # 滞后1到7天
    lagged_formula = "SentimentScore ~ HeatCount"
    for i in range(1, lag_days + 1):
        lagged_formula += f" + HeatCount_lag{i}"  # 每天滞后
    model_name = f"滞后{lag_days}天模型"
    print(f"运行包含滞后{lag_days}天的模型")
    mlik = run_inla_model(lagged_formula, df, model_name)
    mliks.append(mlik)
    models.append(model_name)

# 绘制边际对数似然值变化趋势
plt.figure(figsize=(12, 6))
plt.plot(models, mliks, marker='o')
plt.xticks(rotation=45)
plt.ylabel('边际对数似然值')
plt.title('模型边际对数似然值变化趋势')
plt.show()

运行不包含滞后变量的模型


/usr/local/lib/python3.10/dist-packages/rpy2/robjects/pandas2ri.py:55: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for name, values in obj.iteritems():



 *** inla.core.safe:  The inla program failed, but will rerun in case better initial values may help. try=1/1 


  object 'SentimentScore' not found
The inla program failed and the maximum number of tries has been reached.



RRuntimeError: Error in inla.core.safe(formula = formula, family = family, contrasts = contrasts,  : 
  object 'SentimentScore' not found
The inla program failed and the maximum number of tries has been reached.
